In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import gc
import warnings
warnings.filterwarnings('ignore')

#This code was reference By Boris Ilin 

In [2]:
sales_train_evaluation=pd.read_csv('data/sales_train_evaluation.csv')
sales_train_evaluation.head(3)

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,4,0,0,0,0,3,3,0,1
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,1,2,1,1,0,0,0,0,0
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,2,0,0,0,2,3,0,1


In [3]:
timesteps=14
startDay=350

In [4]:
def downcase(df):  
    float_cols=[c for c in df if df[c].dtype == "float64" ] 
    int_cols = [c for c in df if df[c].dtype in ["int64","int32"]]
    df[float_cols]= df[float_cols].astype(np.float32)
    df[int_cols] = df[int_cols].astype(np.int16)
    return df

In [5]:
sales_train_evaluation=downcase(sales_train_evaluation)

In [6]:
#transpose that table
sales_train_evaluation= sales_train_evaluation.T
sales_train_evaluation.head(8)


,0,1,2,3,4,5,6,7,8,9,...,30480,30481,30482,30483,30484,30485,30486,30487,30488,30489
id,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_006_CA_1_evaluation,HOBBIES_1_007_CA_1_evaluation,HOBBIES_1_008_CA_1_evaluation,HOBBIES_1_009_CA_1_evaluation,HOBBIES_1_010_CA_1_evaluation,...,FOODS_3_818_WI_3_evaluation,FOODS_3_819_WI_3_evaluation,FOODS_3_820_WI_3_evaluation,FOODS_3_821_WI_3_evaluation,FOODS_3_822_WI_3_evaluation,FOODS_3_823_WI_3_evaluation,FOODS_3_824_WI_3_evaluation,FOODS_3_825_WI_3_evaluation,FOODS_3_826_WI_3_evaluation,FOODS_3_827_WI_3_evaluation
item_id,HOBBIES_1_001,HOBBIES_1_002,HOBBIES_1_003,HOBBIES_1_004,HOBBIES_1_005,HOBBIES_1_006,HOBBIES_1_007,HOBBIES_1_008,HOBBIES_1_009,HOBBIES_1_010,...,FOODS_3_818,FOODS_3_819,FOODS_3_820,FOODS_3_821,FOODS_3_822,FOODS_3_823,FOODS_3_824,FOODS_3_825,FOODS_3_826,FOODS_3_827
dept_id,HOBBIES_1,HOBBIES_1,HOBBIES_1,HOBBIES_1,HOBBIES_1,HOBBIES_1,HOBBIES_1,HOBBIES_1,HOBBIES_1,HOBBIES_1,...,FOODS_3,FOODS_3,FOODS_3,FOODS_3,FOODS_3,FOODS_3,FOODS_3,FOODS_3,FOODS_3,FOODS_3
cat_id,HOBBIES,HOBBIES,HOBBIES,HOBBIES,HOBBIES,HOBBIES,HOBBIES,HOBBIES,HOBBIES,HOBBIES,...,FOODS,FOODS,FOODS,FOODS,FOODS,FOODS,FOODS,FOODS,FOODS,FOODS
store_id,CA_1,CA_1,CA_1,CA_1,CA_1,CA_1,CA_1,CA_1,CA_1,CA_1,...,WI_3,WI_3,WI_3,WI_3,WI_3,WI_3,WI_3,WI_3,WI_3,WI_3
state_id,CA,CA,CA,CA,CA,CA,CA,CA,CA,CA,...,WI,WI,WI,WI,WI,WI,WI,WI,WI,WI
d_1,0,0,0,0,0,0,0,12,2,0,...,0,14,1,0,4,0,0,0,0,0
d_2,0,0,0,0,0,0,0,15,0,0,...,0,11,1,0,4,0,0,6,0,0


In [7]:
sales_train_evaluation = sales_train_evaluation[6 + startDay:]
sales_train_evaluation.head(5)

,0,1,2,3,4,5,6,7,8,9,...,30480,30481,30482,30483,30484,30485,30486,30487,30488,30489
d_351,0,0,0,2,0,0,0,24,3,2,...,0,9,1,0,11,0,0,1,0,0
d_352,0,0,0,0,0,0,0,9,0,2,...,0,5,4,0,8,0,1,2,0,0
d_353,0,0,0,4,2,0,0,2,1,1,...,0,15,2,0,3,0,1,2,0,0
d_354,0,1,0,2,0,0,0,7,1,0,...,0,5,1,0,3,0,0,0,0,0
d_355,0,0,0,1,2,0,0,0,0,0,...,0,7,1,0,1,0,1,1,0,0


In [8]:
from sklearn.preprocessing import MinMaxScaler

sc= MinMaxScaler(feature_range=(0,1))
sales_train_evaluation_scaled=sc.fit_transform(sales_train_evaluation)

In [9]:
sales_train_evaluation

,0,1,2,3,4,5,6,7,8,9,...,30480,30481,30482,30483,30484,30485,30486,30487,30488,30489
d_351,0,0,0,2,0,0,0,24,3,2,...,0,9,1,0,11,0,0,1,0,0
d_352,0,0,0,0,0,0,0,9,0,2,...,0,5,4,0,8,0,1,2,0,0
d_353,0,0,0,4,2,0,0,2,1,1,...,0,15,2,0,3,0,1,2,0,0
d_354,0,1,0,2,0,0,0,7,1,0,...,0,5,1,0,3,0,0,0,0,0
d_355,0,0,0,1,2,0,0,0,0,0,...,0,7,1,0,1,0,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
d_1937,0,0,0,1,0,0,1,5,0,1,...,3,6,3,0,0,1,0,1,0,0
d_1938,3,0,2,3,0,0,0,4,0,1,...,1,4,3,1,2,0,1,0,1,2
d_1939,3,0,3,0,2,5,1,1,0,0,...,3,4,3,1,1,0,0,1,1,2
d_1940,0,0,0,2,1,2,1,40,1,0,...,0,1,0,0,3,1,1,0,1,5


In [10]:
X_train= []
y_train= []

for i in range(timesteps,1941-startDay):
    X_train.append(sales_train_evaluation_scaled[i-timesteps:i])
    y_train.append(sales_train_evaluation_scaled[i][0:30490])

In [11]:
del sales_train_evaluation_scaled

In [12]:
X_train= np.array(X_train)
y_train= np.array(y_train)

print (X_train.shape)
print (y_train.shape)

(1577, 14, 30490)
(1577, 30490)


In [14]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout

#Initialising the RNN
regerssor = Sequential()

#Adding the first LSTM layer and some Dropout regularisation

layer_1_units=50

regerssor.add(LSTM(units= layer_1_units, return_sequences=False, input_shape=(X_train.shape[1],X_train.shape[2])))

#Adding Dropout regularisation

regerssor.add(Dropout(0.2))

#Adding out layer
regerssor.add(Dense(units=30490))

#compiling the RNN
regerssor.compile(optimizer= 'adam', loss = 'mean_squared_error')

#Fitting the RNN to the Traning set.
epoch_no=32
batch_size_RNN=44
regerssor.fit(X_train,y_train,epochs=epoch_no,batch_size=batch_size_RNN)


Epoch 1/32
36/36 [==============================] - 28s 622ms/step - loss: 0.0185
Epoch 2/32
36/36 [==============================] - 21s 591ms/step - loss: 0.0162
Epoch 3/32
36/36 [==============================] - 21s 579ms/step - loss: 0.0156
Epoch 4/32
36/36 [==============================] - 22s 597ms/step - loss: 0.0155
Epoch 5/32
36/36 [==============================] - 21s 574ms/step - loss: 0.0153
Epoch 6/32
36/36 [==============================] - 20s 568ms/step - loss: 0.0152
Epoch 7/32
36/36 [==============================] - 21s 570ms/step - loss: 0.0151
Epoch 8/32
36/36 [==============================] - 21s 572ms/step - loss: 0.0150
Epoch 9/32
36/36 [==============================] - 20s 566ms/step - loss: 0.0149
Epoch 10/32
36/36 [==============================] - 20s 564ms/step - loss: 0.0148
Epoch 11/32
36/36 [==============================] - 21s 579ms/step - loss: 0.0148
Epoch 12/32
36/36 [==============================] - 20s 566ms/step - loss: 0.0147
Epoch 13/32
3

In [23]:
evaluation_result= regerssor.evaluate(X_train[:20],y_train[:20],batch_size=batch_size_RNN)


1/1 [==============================] - 0s 302ms/step - loss: 0.0117


In [25]:
evaluation_result

0.01166099775582552

In [24]:
test_accuracy = evaluation_result
print("Test Accuracy:", test_accuracy)

Test Accuracy: 0.01166099775582552


In [18]:
X_train[:20].shape
y_train[:20].shape

(20, 30490)

In [ ]:
#set last 14 day to predict
inputs=sales_train_evaluation[-timesteps:]
inputs= sc.transform(inputs)

In [ ]:
inputs.shape

(14, 30490)

In [ ]:
X_test=[]
X_test.append(inputs[0:timesteps])
X_test = np.array(X_test)
predictions = []


In [ ]:
for j in range(timesteps,timesteps+28):
    predicted_sold=regerssor.predict(X_test[0,j-timesteps:j].reshape(1,timesteps,30490))
    #test_input= np.column_stack(np.array(predicted_sold))
    print (predicted_sold)
    X_test = np.append(X_test,np.array(predicted_sold)).reshape(1,j+1,30490)
    predicted_sold=sc.inverse_transform(np.array(predicted_sold))[:,0:30490]
    predictions.append(predicted_sold)

1/1 [==============================] - 1s 631ms/step
[[0.12102716 0.07186092 0.07699595 ... 0.06083823 0.09210219 0.09627593]]
1/1 [==============================] - 0s 38ms/step
[[0.11021633 0.06775536 0.06894499 ... 0.05793243 0.08024956 0.08631881]]
1/1 [==============================] - 0s 37ms/step
[[0.09595428 0.06402519 0.05737579 ... 0.06340352 0.06402354 0.0784387 ]]
1/1 [==============================] - 0s 37ms/step
[[0.06402862 0.056723   0.03113452 ... 0.08058764 0.02792724 0.0640032 ]]
1/1 [==============================] - 0s 40ms/step
[[0.05800199 0.05547047 0.02678959 ... 0.08153194 0.02178819 0.06096596]]
1/1 [==============================] - 0s 40ms/step
[[0.05763594 0.05551335 0.02667532 ... 0.08099426 0.02163251 0.06056339]]
1/1 [==============================] - 0s 38ms/step
[[0.05758021 0.05550557 0.02660617 ... 0.08098768 0.02161046 0.06037479]]
1/1 [==============================] - 0s 38ms/step
[[0.05757578 0.05549238 0.02656059 ... 0.08103861 0.02161645 0.06

In [ ]:
submission = pd.DataFrame(data=np.array(predictions).reshape(28,30490))
submission = submission.T

submission = pd.concat((submission,submission),ignore_index=True)

sample_submission = pd.read_csv('data/sample_submission.csv')

idColumn = sample_submission[["id"]]

submission[["id"]]= idColumn

cols=list(submission.columns)
cols=cols[-1:] + cols[:-1]

submission = submission[cols]

colsdeneme= ["id"]+[f"F{i}" for i in range(1,29)]

submission.columns = colsdeneme

In [ ]:
submission

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,HOBBIES_1_001_CA_1_validation,0.605136,0.551082,0.479771,0.320143,0.290010,0.288180,0.287901,0.287879,0.287978,...,0.290099,0.290135,0.290164,0.290188,0.290211,0.290231,0.290250,0.290266,0.290281,0.290293
1,HOBBIES_1_002_CA_1_validation,0.359305,0.338777,0.320126,0.283615,0.277352,0.277567,0.277528,0.277462,0.277396,...,0.276761,0.276752,0.276742,0.276734,0.276726,0.276718,0.276711,0.276705,0.276700,0.276696
2,HOBBIES_1_003_CA_1_validation,0.461976,0.413670,0.344255,0.186807,0.160738,0.160052,0.159637,0.159364,0.159178,...,0.158403,0.158402,0.158398,0.158393,0.158388,0.158384,0.158379,0.158376,0.158372,0.158370
3,HOBBIES_1_004_CA_1_validation,1.384127,1.243191,1.222569,1.265233,1.260236,1.259337,1.259797,1.260296,1.260859,...,1.266821,1.266929,1.267005,1.267069,1.267127,1.267180,1.267228,1.267270,1.267307,1.267340
4,HOBBIES_1_005_CA_1_validation,1.069707,0.998703,0.938822,0.819553,0.793474,0.793389,0.793927,0.794200,0.794360,...,0.794917,0.794914,0.794915,0.794917,0.794919,0.794922,0.794924,0.794926,0.794927,0.794929
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60975,FOODS_3_823_WI_3_evaluation,0.497368,0.482016,0.447519,0.359677,0.339524,0.337288,0.337084,0.337145,0.337308,...,0.339834,0.339873,0.339907,0.339940,0.339969,0.339997,0.340021,0.340043,0.340063,0.340079
60976,FOODS_3_824_WI_3_evaluation,0.207234,0.205393,0.218676,0.256980,0.258898,0.258177,0.258576,0.258831,0.258945,...,0.258488,0.258477,0.258469,0.258461,0.258453,0.258445,0.258439,0.258433,0.258428,0.258423
60977,FOODS_3_825_WI_3_evaluation,0.425868,0.405527,0.443825,0.564113,0.570724,0.566960,0.566914,0.567270,0.567743,...,0.572973,0.573059,0.573128,0.573188,0.573243,0.573293,0.573338,0.573378,0.573413,0.573443
60978,FOODS_3_826_WI_3_evaluation,1.105226,0.962995,0.768282,0.335127,0.261458,0.259590,0.259326,0.259397,0.259620,...,0.262931,0.263006,0.263058,0.263102,0.263141,0.263176,0.263207,0.263235,0.263260,0.263281


In [ ]:
submission.to_csv("submission_LSTM.csv",index=False)

In [ ]:
inputs=sales_train_evaluation[-timesteps-28:-14]
inputs= sc.transform(inputs)
inputs.shape

(28, 30490)

In [ ]:
y_test=sc.inverse_transform(y_train[-28:])


In [ ]:
y_test

array([[0., 0., 0., ..., 0., 1., 0.],
       [0., 1., 0., ..., 0., 3., 0.],
       [0., 0., 1., ..., 1., 0., 0.],
       ...,
       [3., 0., 3., ..., 1., 1., 2.],
       [0., 0., 0., ..., 0., 1., 5.],
       [1., 0., 1., ..., 2., 0., 1.]])

In [ ]:
X_test=[]
X_test.append(inputs[0:timesteps]) 
X_test = np.array(X_test)
predictions = []

In [ ]:
X_test.shape

(1, 14, 30490)

In [ ]:
X_test=[]
X_test.append(inputs[0:timesteps])
X_test = np.array(X_test)
predictions = []

In [ ]:
X_test.shape

(1, 14, 30490)

In [ ]:
residaul_sum=[]
for j in range(timesteps,timesteps+28):
    #print (j-timesteps)
    predicted_sold=regerssor.predict(X_test[0,j-timesteps:j].reshape(1,timesteps,30490))
    test_input= np.column_stack(np.array(predicted_sold))
    X_test = np.append(X_test,np.array(predicted_sold)).reshape(1,j+1,30490)
    predicted_sold=sc.inverse_transform(np.array(predicted_sold))[:,0:30490]
    residaul=predicted_sold-y_test[j-timesteps]
    print (np.sum(residaul**2))
    residaul_sum.append(np.sum(residaul**2))

1/1 [==============================] - 0s 57ms/step


116902.21242574566
1/1 [==============================] - 0s 68ms/step
95678.30028978965
1/1 [==============================] - 0s 38ms/step
101901.92168618579
1/1 [==============================] - 0s 36ms/step
110318.81462111084
1/1 [==============================] - 0s 38ms/step
180744.02427966657
1/1 [==============================] - 0s 38ms/step
253984.0947063389
1/1 [==============================] - 0s 41ms/step
250921.96971639307
1/1 [==============================] - 0s 52ms/step
180377.64901564072
1/1 [==============================] - 0s 44ms/step
199149.52458355582
1/1 [==============================] - 0s 37ms/step
164913.58531305625
1/1 [==============================] - 0s 42ms/step
163407.64905071774
1/1 [==============================] - 0s 37ms/step
228062.78428822325
1/1 [==============================] - 0s 38ms/step
322694.79564887367
1/1 [==============================] - 0s 40ms/step
222195.28905531368
1/1 [==============================] - 0s 37ms/step
227842.2

In [ ]:
mse=np.sum(residaul_sum)/(28*30490)
rmse=mse**0.5
print (rmse)

2.535312883636704
